Downloading Library

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 31.6 MB/s eta 0:00:00


Cloning ultralytics repo

In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17075 (delta 19), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17075/17075), 15.69 MiB | 11.52 MiB/s, done.
Resolving deltas: 100% (11724/11724), done.
/content/yolov5


Uploading the video file

In [4]:
from google.colab import files

uploaded = files.upload()


Saving video.mp4 to video.mp4


Tracking and calculating speed using Yolo v5
the speed is calculated in Pixels/seconde

In [5]:
import cv2
import torch
import numpy as np

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Use YOLOv5s (small model)

# Video input and output settings
input_video = list(uploaded.keys())[0]
video = cv2.VideoCapture(input_video)

fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_file = 'output.avi'
output = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Initialize variables for speed calculation
prev_position = None
speed = 0.0

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Run YOLOv5 detection
    results = model(frame)
    detections = results.pandas().xyxy[0]  # Get detections as a pandas DataFrame

    for _, row in detections.iterrows():
        # Filter for the tennis ball (YOLO class: 'sports ball')
        if row['name'] == 'sports ball':
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            conf = row['confidence']

            # Draw bounding box and confidence
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Speed (Pixel/s): {speed:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Calculate the ball's center
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

            # Calculate speed if we have a previous position
            if prev_position is not None:
                displacement = np.sqrt((center_x - prev_position[0])**2 + (center_y - prev_position[1])**2)
                speed = displacement * fps  # Speed in pixels per second

            # Update previous position
            prev_position = (center_x, center_y)

    # Write the processed frame to the output video
    output.write(frame)

# Release resources
video.release()
output.release()

# Download the processed video
files.download(output_file)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-12-20 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 106MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

En utilisant le filtre de kalman et le model Yolo5

In [10]:


output_file = 'output_with_kalman.avi'
output = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Initialize Kalman filter
kalman = cv2.KalmanFilter(4, 2)  # 4 states (x, y, dx, dy) and 2 measurements (x, y)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                      [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                     [0, 1, 0, 1],
                                     [0, 0, 1, 0],
                                     [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.05

prev_position = None
trajectory_points = []  # To store the trajectory for visualization

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Run YOLOv5 detection
    results = model(frame)
    detections = results.pandas().xyxy[0]  # Get detections as a pandas DataFrame

    observed_position = None

    for _, row in detections.iterrows():
        # Filter for the tennis ball (YOLO class: 'sports ball')
        if row['name'] == 'sports ball':
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            conf = row['confidence']

            # Calculate the ball's center
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            observed_position = np.array([[np.float32(center_x)], [np.float32(center_y)]])

            # Draw bounding box and confidence
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Conf: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

    # Update Kalman filter
    if observed_position is not None:
        kalman.correct(observed_position)  # Update with observed position
    predicted_position = kalman.predict()  # Predict next position

    # Extract predicted coordinates
    pred_x, pred_y = int(predicted_position[0]), int(predicted_position[1])

    # Add to trajectory points
    trajectory_points.append((pred_x, pred_y))

    # Draw trajectory
    for i in range(1, len(trajectory_points)):
        cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (255, 0, 0), 2)

    # Annotate predicted position
    cv2.circle(frame, (pred_x, pred_y), 5, (255, 0, 0), -1)
    cv2.putText(frame, f"Predicted: ({pred_x}, {pred_y})", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the processed frame to the output video
    output.write(frame)

# Release resources
video.release()
output.release()

# Download the processed video
files.download(output_file)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-20 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
<ipython-input-10-6adc05aaf730>:66: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(predicted_position[0]), int(predicted_position[1])
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with am

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-3xyiibnp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-3xyiibnp
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.6 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

Using detectron

In [4]:
from google.colab import files
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo


# Upload video file
uploaded = files.upload()
input_video = list(uploaded.keys())[0]

# Detectron2 setup
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for predictions
cfg.MODEL.DEVICE = "cuda"

predictor = DefaultPredictor(cfg)

# Video input and output
video = cv2.VideoCapture(input_video)
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_file = "output_detectron.avi"
output = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Map COCO class IDs to names
COCO_CLASSES = {35: "sports ball"}

trajectory_points = []  # Store the ball's trajectory

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Run Detectron2 inference
    outputs = predictor(frame)
    instances = outputs["instances"]

    # Filter for "sports ball" class
    ball_indices = (instances.pred_classes.cpu().numpy() == 37)
    ball_boxes = instances.pred_boxes[ball_indices].tensor.cpu().numpy()

    for box in ball_boxes:
        x1, y1, x2, y2 = map(int, box)

        # Calculate the ball's center
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # Draw bounding box and center point
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
        cv2.putText(frame, "Tennis Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Update trajectory
        trajectory_points.append((center_x, center_y))

    # Draw trajectory
    for i in range(1, len(trajectory_points)):
        cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (255, 0, 0), 2)

    # Write the frame to the output video
    output.write(frame)

# Release resources
video.release()
output.release()

# Download the processed video
files.download(output_file)


Saving video.mp4 to video (1).mp4


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>